Testing notebook to run my code in

In [4]:
#import necessary packages
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
from sklearn import preprocessing as pre
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split 
from sklearn.cluster import KMeans 
from sklearn.model_selection import cross_val_score
df = pd.read_csv('terrorismp3.csv')
target = 'affiliation'
df.head()


,name,age,date,where,region,affiliation,role,incrimination,circumstances,method,offenses,terrorist
0,Fu'ad Abed,26,1/2/2023,Jenin,Judea-Samaria,Hamas,Militant,Official martyrdom poster. pictures with rifle...,Clashes with IDF demolishing the house of a te...,Fire,1,1
1,Muhammad Khushiyeh,22,1/2/2023,Jenin,Judea-Samaria,al-Aqsa's Martyrs Brigades,Militant,Official martyrdom poster. pictures with rifles,Clashes with IDF demolishing the house of a te...,Fire,1,1
2,Adam Ayyad,15,1/3/2023,Dheisheh,Judea-Samaria,PFLP,Militant,Official martyrdom poster. his personal will. ...,Clashes with IDF forces making arrests in the ...,Fire,1,1
3,Amer Abu Zaytun,16,1/5/2023,Nablus,Judea-Samaria,al-Aqsa's Martyrs Brigades,Militant,Official martyrdom poster. pictures with rifle...,Clashes with IDF forces making arrests in the ...,Fire,1,1
4,Abd al-Hadi Fakhri Youssef Nazzal,18,1/12/2023,Jenin,Judea-Samaria,PIJ,Militant,Official martyrdom poster. pictures with rifle...,Clashes with IDF forces making arrests in the ...,Fire,1,1


In [13]:
df.head()

,name,age,date,where,region,affiliation,role,incrimination,circumstances,method,offenses,terrorist
0,Fu'ad Abed,26,1/2/2023,Jenin,Judea-Samaria,Hamas,Militant,Official martyrdom poster. pictures with rifle...,Clashes with IDF demolishing the house of a te...,Fire,1,1
1,Muhammad Khushiyeh,22,1/2/2023,Jenin,Judea-Samaria,al-Aqsa's Martyrs Brigades,Militant,Official martyrdom poster. pictures with rifles,Clashes with IDF demolishing the house of a te...,Fire,1,1
2,Adam Ayyad,15,1/3/2023,Dheisheh,Judea-Samaria,PFLP,Militant,Official martyrdom poster. his personal will. ...,Clashes with IDF forces making arrests in the ...,Fire,1,1
3,Amer Abu Zaytun,16,1/5/2023,Nablus,Judea-Samaria,al-Aqsa's Martyrs Brigades,Militant,Official martyrdom poster. pictures with rifle...,Clashes with IDF forces making arrests in the ...,Fire,1,1
4,Abd al-Hadi Fakhri Youssef Nazzal,18,1/12/2023,Jenin,Judea-Samaria,PIJ,Militant,Official martyrdom poster. pictures with rifle...,Clashes with IDF forces making arrests in the ...,Fire,1,1


In [2]:
# need to create dummy binary variables so that model can interpret values 
#need to scale numerical data 0-1
#create object to represent categorical features in dataset
categorical_variables = [i for i in df.select_dtypes(include = object) if i != target and i != 'name' and i != 'incrimination' and i != 'circumstances' and i != 'date' and i != 'where']
categorical_variables.append('offenses')
categorical_variables.append('terrorist')
categorical_variables

['region', 'role', 'method', 'offenses', 'terrorist']

In [37]:
dummy_variables = pd.get_dummies(df[categorical_variables])
dummy_variables

,offenses,terrorist,region_Gaza Strip,region_Israel,region_Judea-Samaria,region_Syria,role_Commander,role_Militant,role_Military Engineer,role_Officer,role_Operative,role_Secretary,role_Senior dentist,role_Senior official,role_Terrorist associate,role_Terrorist family,method_Airstrike,method_Failed PL Attack,method_Fire,method_Injury
0,1,1,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False
1,1,1,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False
2,1,1,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False
3,1,1,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False
4,1,1,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,0,0,True,False,False,False,False,False,False,False,False,False,False,False,False,True,True,False,False,False
79,0,0,True,False,False,False,False,False,False,False,False,False,False,False,True,False,True,False,False,False
80,0,0,True,False,False,False,False,False,False,False,False,False,False,False,False,True,True,False,False,False
81,3,1,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False
